In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*
__author__="Matthew Nicklay, Tim Guo, Chris Barrington-Leigh"
__license__="MIT license"
from ast import literal_eval
from pandas import DataFrame  # http://github.com/pydata/pandas
import re
import os
import requests               # http://github.com/kennethreitz/requests
import subprocess
import sys
import pickle

corpora = dict(eng_us_2012=17, eng_us_2009=5, eng_us_2019=28,
               eng_gb_2012=18, eng_gb_2009=6, eng_gb_2019=26,
               chi_sim_2019=34, chi_sim_2012=23, chi_sim_2009=11,
               eng_2012=15, eng_2009=0,
               eng_fiction_2012=16, eng_fiction_2009=4, eng_1m_2009=1,
               fre_2019=30, fre_2012=19, fre_2009=7,
               ger_2019=31, ger_2012=20, ger_2009=8,
               heb_2012=24,
               heb_2009=9,
               spa_2019=32, spa_2012=21, spa_2009=10,
               rus_2019=36, rus_2012=25, rus_2009=12,
               ita_2019=33, ita_2012=22)


def getNgrams(query, corpus='eng_us_2019', startYear=1500, endYear=2019, smoothing=None, caseInsensitive=False,
              filename=None
              ):
    params = dict(content=query, year_start=startYear, year_end=endYear,
                  corpus=corpora[corpus], smoothing=smoothing,
                  case_insensitive=caseInsensitive)
    if params['case_insensitive'] is False:
        params.pop('case_insensitive')
    if '?' in params['content']:
        params['content'] = params['content'].replace('?', '*')
    if '@' in params['content']:
        params['content'] = params['content'].replace('@', '=>')

    req = requests.get('http://books.google.com/ngrams/graph', params=params)
    res = re.findall('ngrams.data = .*\];', req.text)
    if not len(res)==1: print(req.text)
    assert(len(res)==1)

    if res:
        dataDict = literal_eval(res[0].replace(
            "ngrams.data = ", "").replace(";", ""))
        data = {qry['ngram']: qry['timeseries']
                for qry in dataDict}
        df = DataFrame(data)
        df.insert(0, 'year', list(range(startYear, endYear + 1)))
    else:
        df = DataFrame()
    return df

# 

In [2]:
# import pandas as pd
# df = pd.read_csv('/Users/isabellelorge/Desktop/df_lex_entail_new.csv')

In [40]:
# freq = {}

In [65]:
# for s in range(0,len(df)+1, 10):
#     print(s)
#     df_t = df['target1'].loc[s:s+10]
#     t = [i for i in df_t if i not in freq]
#     print(t)
#     if t != []:
#         df_gram = getNgrams(','.join(t), startYear=2000, endYear=2015)
#         print(df_gram)
#         for w in t:
#             print(w)
#             w1 = w
#             if '-' in w:
#                 w1 = ' - '.join(w.split('-'))
#             f = df_gram.mean()[w1]
#             freq[w] = f

In [58]:
import json
import json
  
# Opening JSON file
f = open('/Users/isabellelorge/Desktop/exp_freq.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)

In [59]:
type(list(data.keys())[0])

str

In [60]:
from ast import literal_eval
l = [literal_eval(k) for k in list(data.keys()) if data[k] == 0 ]

In [61]:
l

[('occasionally', 'true'),
 ('occasionally', 'criminal'),
 ('occasionally', 'correct'),
 ('occasionally', 'corrupt'),
 ('occasionally', 'helpful'),
 ('occasionally', 'wrong'),
 ('occasionally', 'violent'),
 ('occasionally', 'coherent'),
 ('occasionally', 'useful'),
 ('occasionally', 'entertaining'),
 ('occasionally', 'inexplicable'),
 ('occasionally', 'wilfull'),
 ('occasionally', 'crazy'),
 ('occasionally', 'interesting'),
 ('occasionally', 'incorrect'),
 ('occasionally', 'dead'),
 ('occasionally', 'military'),
 ('occasionally', 'worrying'),
 ('occasionally', 'reasonable'),
 ('occasionally', 'right'),
 ('occasionally', 'lethal'),
 ('occasionally', 'unreliable'),
 ('occasionally', 'painful'),
 ('occasionally', 'trivial'),
 ('occasionally', 'biased'),
 ('occasionally', 'good'),
 ('occasionally', 'overblown'),
 ('occasionally', 'controversial'),
 ('occasionally', 'illegal'),
 ('occasionally', 'bankrupt'),
 ('occasionally', 'funny'),
 ('occasionally', 'necessary'),
 ('occasionally', 'over

In [30]:
str(l[0][0]+ ' '+ l[0][1])

'occasionally true'

In [62]:
import numpy as np

for i in range(0, len(l), 10):
    b = l[i:i + 10]
    print(b)
    df = getNgrams(', '.join([str(e[0]+ ' '+ e[1]) for e in b]))
    print(df.mean())
    for e in b:
        try:
            f = np.log(df.mean()[str(e[0]+ ' '+ e[1])])
            if str(e) in data:
                data[str(e)] = f
            else:
                print('NOT KEY?')
        except Exception as er:
            print(er)
            if str(e) in data:
                data[str(e)] = 0
            else:
                print('NOT KEY')
    

[('occasionally', 'true'), ('occasionally', 'criminal'), ('occasionally', 'correct'), ('occasionally', 'corrupt'), ('occasionally', 'helpful'), ('occasionally', 'wrong'), ('occasionally', 'violent'), ('occasionally', 'coherent'), ('occasionally', 'useful'), ('occasionally', 'entertaining')]
year                         1.759500e+03
occasionally true            1.962732e-09
occasionally criminal        1.102064e-10
occasionally correct         7.333682e-10
occasionally corrupt         2.296864e-10
occasionally helpful         1.675666e-09
occasionally wrong           5.969817e-10
occasionally violent         3.370269e-09
occasionally coherent        5.875943e-11
occasionally useful          1.542902e-08
occasionally entertaining    5.722224e-10
dtype: float64
[('occasionally', 'inexplicable'), ('occasionally', 'wilfull'), ('occasionally', 'crazy'), ('occasionally', 'interesting'), ('occasionally', 'incorrect'), ('occasionally', 'dead'), ('occasionally', 'military'), ('occasionally', 'wo

In [63]:
[k for k in data if data[k] == 0]

["('occasionally', 'wilfull')",
 "('arguably', 'debatable')",
 "('arguably', 'MORE')",
 "('arguably', 'greedy')",
 "('arguably', 'idiotic')",
 "('arguably', 'lamentable')",
 "('arguably', 'awful')",
 "('arguably', 'intelligible')",
 "('arguably', 'calmer')",
 "('arguably', 'invaluable')",
 "('arguably', 'conformist')",
 "('arguably', 'passe')"]

In [65]:
with open('/Users/isabellelorge/Desktop/exp_freq.json', 'w') as fp:
    json.dump(data, fp)

In [41]:
adjectives

{' never': {'count': 1342,
  'adj': {'last': 17,
   'accurate': 13,
   'wrong': 142,
   'good': 78,
   'irrelevant': 2,
   'true': 51,
   'socialist': 15,
   'bad': 22,
   'obvious': 3,
   'necessary': 26,
   'popular': 10,
   'great': 6,
   'underwent': 1,
   'idle': 1,
   'private': 3,
   'possible': 10,
   'justifiable': 9,
   'persistent': 3,
   'homeless': 1,
   'flat': 1,
   'own': 2,
   'incompatible': 1,
   'free': 31,
   'stable': 6,
   'achievable': 2,
   'rich': 4,
   'embarrassed': 2,
   'equal': 16,
   'perfect': 20,
   'vacuous': 1,
   'healthy': 4,
   'ummed': 1,
   'proper': 2,
   'pretty': 5,
   'progressive': 2,
   'recoverable': 2,
   'greener': 1,
   'illegal': 8,
   'profitable': 4,
   'insolvent': 2,
   'neutral': 7,
   'equivalent': 1,
   'moral': 3,
   'uploaded': 1,
   'satisfied': 23,
   'legitimate': 13,
   'captive': 1,
   'happy': 30,
   'acceptable': 45,
   'constructive': 2,
   'higher': 4,
   'realistic': 7,
   'communist': 9,
   'static': 7,
   'right':

In [15]:
import json
j = '/Users/isabellelorge/Desktop/adjectives.json'
with open(j, 'r') as f:
    adjectives = json.load(f)

In [18]:
[(k, len(adjectives[k]['adj'])) for k in adjectives]

[(' never', 334),
 ('sometimes', 165),
 ('usually', 338),
 ('generally', 441),
 ('often', 434),
 ('already', 475),
 ('frequently', 61),
 ('always', 799),
 ('maybe', 113),
 ('perhaps', 64),
 ('possibly', 156),
 ('probably', 451),
 ('really', 1311),
 ('actually', 900),
 ('certainly', 406),
 ('definitely', 579),
 ('not', 3231),
 ('hardly', 471),
 ('basically', 305),
 ('quite', 1508),
 ('pretty', 1995),
 ('very', 2767),
 ('seriously', 235),
 ('completely', 1238)]

In [ ]:
import numpy as np
import time

adj_freq = adjectives.copy()

def query_api(adj_list, start):
    query =  ','.join([k + ' ' + a for a in adj_list[start: start + 12]])
    url, urlquery, df = getNgrams(query, startYear=2000, endYear = 2015)
    s = df.sum()
    print(s)
    for a in adj_list[start: start+12]:
        if k.strip() + ' ' + a in s.index:
            freq = s[k.strip() + ' ' + a]
            # ! the more negative/bigger the absolute value, the less frequent
            adj_freq[k]['adj'][a] = np.log(freq)
        else:
            adj_freq[k]['adj'][a] = 0


for k in adjectives:
    adj_list = list(adjectives[k]['adj'].keys())
    print(len(adj_list))
    for start in range(0, len(adj_list), 12):
        print(start)
        success = False
        count = 1
        sleep = 60
        while not success:
            try:
                query_api(adj_list, start)
                success = True
                count = 1
            except AssertionError:
                    print(f'sleeping for {sleep*count}')
                    time.sleep(sleep*count)
                    count+=1

        
    

334
0
year                3.212000e+04
never last          9.692073e-07
never accurate      6.331090e-08
never wrong         1.493023e-06
never good          2.459927e-06
never irrelevant    3.807730e-08
never true          9.075470e-07
never socialist     3.422221e-09
never bad           2.460146e-07
never obvious       9.683518e-08
never necessary     5.197590e-07
never popular       3.443266e-07
never great         2.076392e-07
dtype: float64
12
year                  3.212000e+04
never underwent       1.071175e-07
never idle            3.088519e-07
never private         5.322807e-08
never possible        1.003189e-06
never justifiable     1.206500e-07
never persistent      3.719172e-09
never homeless        3.666568e-08
never flat            5.438389e-08
never own             5.886813e-07
never incompatible    8.187457e-09
never free            1.130243e-06
never stable          2.151786e-07
dtype: float64
24
year                 3.212000e+04
never achievable     2.723764e-08
never 

year                3.212000e+04
never brave         5.034302e-08
never visible       2.203035e-07
never explicit      1.067715e-07
never spiteful      1.001984e-08
never political     3.785319e-08
never charitable    3.550389e-09
never specific      5.966713e-08
never altruistic    5.356564e-09
never offensive     4.349559e-08
never different     4.481687e-08
never laughable     1.770035e-09
never bankrupt      5.427928e-09
dtype: float64
240
year                3.212000e+04
never verifiable    7.004503e-09
never Republican    9.471051e-10
never stagnant      5.207403e-08
never fantastic     3.316438e-09
never convincing    2.983206e-08
never punitive      1.041414e-08
never public        2.616392e-08
never content       4.993582e-07
never sharp         3.388882e-08
dtype: float64
252
year                  3.212000e+04
never embarrassing    1.297899e-08
never polite          3.090648e-08
never pleased         1.900819e-07
never tolerable       5.156392e-09
never qualified       1.8140

year                       3.212000e+04
sometimes unhelpful        1.684951e-08
sometimes appalling        3.474350e-08
sometimes futile           2.852123e-08
sometimes different        5.169945e-07
sometimes difficult        7.367490e-06
sometimes unpredictable    2.820817e-07
sometimes black            1.993697e-07
sometimes incompatible     1.612625e-07
sometimes ugly             8.957305e-08
sometimes silly            9.897237e-08
sometimes dense            5.599096e-08
sometimes worthwhile       4.141718e-08
dtype: float64
120
year                      3.212000e+04
sometimes accurate        6.552544e-08
sometimes misleading      2.744304e-07
sometimes overwhelming    4.116998e-07
sometimes safe            1.439442e-08
sometimes great           2.241333e-07
sometimes romantic        4.229351e-08
sometimes decent          4.717722e-09
sometimes helpful         1.129720e-06
sometimes worthy          1.270282e-08
sometimes inconvenient    7.178059e-08
sometimes inefficient     3.2048

year                   3.212000e+04
usually incompetent    1.041255e-08
usually ugly           2.485787e-08
usually unfettered     2.029031e-09
usually irrelevant     1.125341e-07
usually binary         4.502623e-08
usually biased         7.572735e-08
usually fruitless      4.525758e-08
usually humane         3.000546e-09
usually horrible       1.107904e-08
usually gross          2.217510e-08
usually ignorable      1.553947e-09
usually poor           5.410203e-07
dtype: float64
156
year                       3.212000e+04
usually dim                1.912122e-08
usually male               6.241121e-07
usually straightforward    4.941460e-07
usually transferable       1.616651e-08
usually last               1.732794e-06
usually dysfunctional      1.415429e-08
usually unchangeable       2.368278e-09
usually masculine          4.792892e-08
usually wealthier          1.359127e-08
usually valid              1.786663e-07
usually conservative       2.086208e-07
usually truthful           8.3017

year                      3.212000e+04
generally swell           7.263823e-09
generally friendly        3.116330e-07
generally insane          4.888239e-09
generally painful         5.272792e-08
generally unattractive    4.074203e-08
generally lower           3.318299e-06
generally wrong           1.245179e-07
generally peaked          2.253453e-08
generally false           8.282121e-08
generally atrocious       4.577670e-09
generally better          2.600396e-06
generally right           2.058516e-07
dtype: float64
24
year                     3.212000e+04
generally skeptical      1.992013e-07
generally great          6.644786e-08
generally terrible       1.761905e-08
generally inaccurate     6.758126e-08
generally interesting    9.439778e-08
generally outrageous     4.305906e-09
generally finite         1.048848e-08
generally insincere      2.108267e-09
generally gullible       1.871102e-09
generally harmful        9.407209e-08
generally fine           2.047412e-07
dtype: float64
36
y

year                     3.212000e+04
generally incompetent    6.011701e-08
generally possible       6.652192e-07
generally cheap          4.248003e-08
generally fractious      1.218841e-09
generally benevolent     7.105720e-08
generally unhappy        1.474816e-07
generally repulsive      1.284813e-08
generally wiser          1.740405e-08
generally intentional    6.139100e-09
generally clean          1.749007e-07
dtype: float64
228
year                    3.212000e+04
generally lenient       2.712211e-08
generally unclear       6.159001e-08
generally silly         8.353988e-09
generally pointless     1.775128e-08
generally underrated    1.994052e-08
generally ephemeral     1.244754e-08
generally sexist        3.491869e-09
generally flawed        1.742170e-08
generally anxious       8.900271e-08
generally obnoxious     3.764998e-08
generally unreliable    1.610101e-07
generally efficient     1.219971e-07
dtype: float64
240
year                       3.212000e+04
generally reliable     

year                      3.212000e+04
generally unregulated     4.330296e-08
generally defunct         1.631419e-09
generally frequent        2.526927e-08
generally avoidable       1.964158e-08
generally bright          6.731932e-08
generally temporary       1.032321e-07
generally prescriptive    1.019676e-08
generally southern        2.014658e-08
generally likable         1.311450e-08
generally unfriendly      3.620688e-08
generally unappealing     1.861890e-08
generally democratic      3.391845e-08
dtype: float64
432
year                      3.212000e+04
generally decent          4.861404e-08
generally unresponsive    9.139587e-08
generally unsaleable      8.596590e-10
generally young           2.496258e-07
generally inconsistent    1.270705e-07
generally unthinkable     5.606775e-09
generally consensual      1.068360e-08
generally equal           6.454109e-07
generally wet             5.389655e-08
dtype: float64
434
0
year                  3.212000e+04
often extreme         3.1360

year                  3.212000e+04
often tyrannical      3.214904e-08
often terrible        1.080940e-07
often horrible        3.722888e-08
often incremental     4.206506e-08
often conservative    1.469669e-07
often nominal         3.434630e-08
often punitive        5.216663e-08
often unique          4.765841e-07
often weak            6.522814e-07
often accurate        1.449871e-07
often sticky          4.023812e-08
often reactionary     5.183443e-08
dtype: float64
192
year                     3.212000e+04
often entertaining       1.539319e-07
often treaded            2.642703e-09
often implausible        2.733922e-08
often interchangeable    2.358913e-07
often popular            1.442545e-07
often correct            3.502888e-07
often dangerous          1.175350e-06
often fit                2.730344e-07
often unrepresented      1.336822e-08
often significant        7.196655e-07
often incomplete         1.017447e-06
often lovely             2.752199e-08
dtype: float64
204
year         

year                 3.212000e+04
often absolute       2.359461e-08
often silent         4.291618e-07
often broader        7.978955e-08
often adorable       1.410466e-09
often odd            5.535457e-08
often dead           1.251928e-07
often unavailable    6.955440e-07
often discrete       4.330089e-08
often predictable    1.543789e-07
often dramatic       5.709415e-07
often coercive       6.665039e-08
often messy          2.321960e-07
dtype: float64
408
year                    3.212000e+04
often private           1.964662e-07
often damaging          1.212420e-07
often smug              8.725168e-09
often interesting       2.009096e-07
often ineligible        2.716087e-08
often quiet             1.714039e-07
often vitriolic         5.190324e-08
often tiny              7.911349e-08
often unbelievable      1.805220e-08
often unkind            4.513229e-08
often interdependent    7.892487e-08
often dubious           1.312861e-07
dtype: float64
420
year                    3.212000e+04
of

year                       3.212000e+04
already successful         6.053193e-07
already imaginable         2.125156e-09
already skeptical          1.136806e-07
already lean               6.637170e-08
already excited            2.286728e-07
already rampant            7.989165e-08
already compulsory         5.935160e-09
already addictive          2.147767e-09
already upset              4.026758e-07
already willing            1.189917e-07
already individualistic    1.753725e-09
dtype: float64
192


In [ ]:
with open('/Users/isabellelorge/Desktop/adjectives_freq.json', 'w') as fp:
    json.dump(adjectives, fp)

In [55]:
adjectives

{' never': {'count': 1342,
  'adj': {'last': 9.692072925813835e-07,
   'accurate': 6.331090453268846e-08,
   'wrong': 1.4930228839408042e-06,
   'good': 2.459927467886776e-06,
   'irrelevant': 3.8077295558930614e-08,
   'true': 9.075470219650177e-07,
   'socialist': 3.4222206048977244e-09,
   'bad': 2.4601455560332603e-07,
   'obvious': 9.683518472900605e-08,
   'necessary': 5.197590265356869e-07,
   'popular': 3.443266289363375e-07,
   'great': 2.0763918065148074e-07,
   'underwent': 0,
   'idle': 0,
   'private': 0,
   'possible': 0,
   'justifiable': 0,
   'persistent': 0,
   'homeless': 0,
   'flat': 0,
   'own': 0,
   'incompatible': 0,
   'free': 0,
   'stable': 0,
   'achievable': 0,
   'rich': 0,
   'embarrassed': 0,
   'equal': 0,
   'perfect': 0,
   'vacuous': 0,
   'healthy': 0,
   'ummed': 0,
   'proper': 0,
   'pretty': 0,
   'progressive': 0,
   'recoverable': 0,
   'greener': 0,
   'illegal': 0,
   'profitable': 0,
   'insolvent': 0,
   'neutral': 0,
   'equivalent': 0,
